In [1]:
from sagemaker import get_execution_role
role = get_execution_role()

In [2]:
pip install awswrangler

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pygeohash

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import sagemaker
import boto3
import awswrangler as wr

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import pygeohash as pgh

In [5]:
import os
import boto3

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

import sagemaker
from sagemaker import get_execution_role
import sagemaker_pyspark

role = get_execution_role()

# Configure Spark to use the SageMaker Spark dependency jars
jars = sagemaker_pyspark.classpath_jars()

classpath = ":".join(sagemaker_pyspark.classpath_jars())

# See the SageMaker Spark Github to learn how to connect to EMR from a notebook instance
spark = (
    SparkSession.builder.config("spark.driver.extraClassPath", classpath)\
    .config('spark.driver.memory','32G')\
    .master("local[*]")\
    .getOrCreate()
)

spark

In [6]:
# https://sagemaker-examples.readthedocs.io/en/latest/sagemaker-spark/pyspark_mnist/pyspark_mnist_kmeans.html#Loading-the-Data
import boto3

cn_regions = ["cn-north-1", "cn-northwest-1"]
region = boto3.Session().region_name
endpoint_domain = "com.cn" if region in cn_regions else "com"
spark._jsc.hadoopConfiguration().set(
    "fs.s3a.endpoint", "s3.{}.amazonaws.{}".format(region, endpoint_domain)
)

In [7]:
from pyspark.sql.functions import *

In [8]:
center_name = 'SEATTLE' #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< CHOOSE!!!
hour_diff = 8 #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< CHOOSE!!!

time_interval = 6
time_lag = "INTERVAL " + str(hour_diff) + " HOURS"
time_lag

'INTERVAL 8 HOURS'

%%time
latlon = spark.read.format('csv')\
        .options(header= 'true', inferSchema= 'true')\
        .load("s3a://partly-cloudy-common-area/proof_of_concept/latlonRAP.csv")

latlon.count()

latlon.show(5)

geoHashFunc = udf(lambda x,y: pgh.encode(x,y, precision= 5))
latlon = latlon.withColumn("geoEncode", geoHashFunc(col('Lat'), col('Lon')))

latlon.show(5)

latlon = latlon.drop("Lat", "Lon")

%%time
centers = spark.read.format('csv')\
        .options(header= 'true', inferSchema= 'true')\
        .load("s3a://partly-cloudy-common-area/proof_of_concept/rapCellNxNy_ARTCC_intersect.csv")

centers.count()

centers.groupBy(col("NAME")).agg(count(col("NAME"))).show()

centers = centers.drop("IDENT")

%%time
rap = spark.read.format('parquet')\
        .load("s3a://partly-cloudy-rap-parquet/*.parquet")


rap.printSchema()

%%time
rap_centers = rap.join(centers, ['Nx', 'Ny'], "inner")
rap_centers.count()

rap_region = rap_centers.filter(col('NAME') == center_name)
rap_region.show()

%%time
rap_flat200430 = rap_region.filter((col("FLevel") >= 200) & (col("FLevel") <= 430))\
                            .groupBy("dateTime", "Nx", "Ny")\
                            .agg(sum("IsISSR").alias("iSSRs200430"),
                                 mean("Temperature").alias("meanTemp200430"),
                                 mean("RH_ice").alias("meanRH_ice200430"))
rap_flat200430.count()

%%time
rap_flat300370 = rap_region.filter((col("FLevel") >= 300) & (col("FLevel") <= 370))\
                            .groupBy("dateTime", "Nx", "Ny")\
                            .agg(sum("IsISSR").alias("iSSRs300370"),
                                 mean("Temperature").alias("meanTemp300370"),
                                 mean("RH_ice").alias("meanRH_ice300370"))
rap_flat300370.count()

centerName = center_name.lower()
path = "s3a://partly-cloudy-common-area/" + centerName + "_flat300370.parquet"
rap_flat300370.write.parquet(path, mode= "overwrite", compression= 'gzip')

In [9]:
%%time
centerName = center_name.lower()
rap_flat200430 = spark.read.format('parquet')\
                        .load("s3a://partly-cloudy-common-area/" + centerName + "_flat200430.parquet")
rap_flat200430.show()

+-------------------+---+---+-----------+------------------+------------------+
|           dateTime| Nx| Ny|iSSRs200430|    meanTemp200430|  meanRH_ice200430|
+-------------------+---+---+-----------+------------------+------------------+
|2021-05-21 06:00:00| 53|132|          0|230.92058333333333| 25.25666666666667|
|2021-05-21 06:00:00| 36|139|          0|231.18616666666665|24.986666666666665|
|2021-05-21 06:00:00| 81|154|          0|231.76424999999998| 43.10083333333333|
|2021-05-21 06:00:00| 37|155|          0|231.26941666666664| 36.97083333333334|
|2021-05-21 06:00:00| 57|158|          0| 230.2693333333333| 34.20166666666667|
|2021-05-21 06:00:00| 37|160|          0|231.34749999999997| 41.57333333333333|
|2021-05-21 06:00:00| 74|168|          0|230.82674999999998|25.049166666666668|
|2021-05-21 06:00:00| 37|170|          0|231.52483333333336|             46.54|
|2021-05-20 07:00:00| 57|132|          0|233.09749999999997| 9.378333333333334|
|2021-05-20 07:00:00| 33|142|          0

In [10]:
%%time
centerName = center_name.lower()
rap_flat300370 = spark.read.format('parquet')\
                        .load("s3a://partly-cloudy-common-area/" + centerName + "_flat300370.parquet")
rap_flat300370.show()

+-------------------+---+---+-----------+------------------+------------------+
|           dateTime| Nx| Ny|iSSRs300370|    meanTemp300370|  meanRH_ice300370|
+-------------------+---+---+-----------+------------------+------------------+
|2021-05-21 06:00:00| 40|134|          0|226.53724999999997|            12.835|
|2021-05-21 06:00:00| 56|136|          0|         229.30275| 4.779999999999999|
|2021-05-21 06:00:00| 53|143|          0|           228.709|            5.9825|
|2021-05-21 06:00:00| 54|148|          0|          228.6465|               6.9|
|2021-05-21 06:00:00| 65|154|          0|           228.834| 4.470000000000001|
|2021-05-21 06:00:00| 82|157|          0|           225.084|26.049999999999997|
|2021-05-21 06:00:00| 42|161|          0|224.25574999999998|           40.2375|
|2021-05-21 06:00:00| 37|166|          0|223.22449999999998|           48.8125|
|2021-05-21 06:00:00| 55|167|          0|227.72474999999997|4.5024999999999995|
|2021-05-21 06:00:00| 42|169|          0

In [11]:
%%time
rap_flat_combined = rap_flat200430.join(rap_flat300370, ['dateTime', 'Nx', 'Ny'], "inner")
rap_flat_combined.show()

+-------------------+---+---+-----------+------------------+------------------+-----------+------------------+------------------+
|           dateTime| Nx| Ny|iSSRs200430|    meanTemp200430|  meanRH_ice200430|iSSRs300370|    meanTemp300370|  meanRH_ice300370|
+-------------------+---+---+-----------+------------------+------------------+-----------+------------------+------------------+
|2020-06-01 00:00:00| 44|173|          3|232.63833333333332| 73.80749999999999|          2|222.78725000000003|            69.685|
|2020-06-01 00:00:00| 54|149|          0| 233.4455833333333| 68.72916666666667|          0|          224.5215| 91.44749999999999|
|2020-06-01 00:00:00| 56|176|          0|233.02383333333333|           34.3625|          0|         223.17775|50.855000000000004|
|2020-06-01 00:00:00| 58|149|          0|233.48208333333332| 68.91416666666667|          0|         224.58375|           94.6525|
|2020-06-01 00:00:00| 62|134|          0|         234.56025|           66.9025|          0

%%time
rap_flat_combined_pdPandas = rap_flat_combined.toPandas()
rap_flat_combined_pdPandas

%%time
rap_flat200430 = rap_flat200430.withColumn("localDateTime", col("dateTime") - expr(time_lag))
rap_flat200430.count()
rap_flat200430 = rap_flat200430.withColumn("dateOnly", to_date(col("localDateTime")))
rap_flat200430 = rap_flat200430.withColumn("hour", hour(col("localDateTime")))
periodFunc = udf(lambda x: x//time_interval * time_interval)
rap_flat200430 = rap_flat200430.withColumn("period", periodFunc(col('hour')))
rap_flat200430.show()

%%time
rap_flat300370 = rap_flat_combined.withColumn("localDateTime", col("dateTime") - expr(time_lag))
rap_flat300370.count()
rap_flat300370 = rap_flat300370.withColumn("dateOnly", to_date(col("localDateTime")))
rap_flat300370 = rap_flat300370.withColumn("hour", hour(col("localDateTime")))
periodFunc = udf(lambda x: x//time_interval * time_interval)
rap_flat300370 = rap_flat300370.withColumn("period", periodFunc(col('hour')))
rap_flat300370.show()

In [12]:
%%time
rap_flat_combined = rap_flat_combined.withColumn("localDateTime", col("dateTime") - expr(time_lag))
rap_flat_combined.count()
rap_flat_combined = rap_flat_combined.withColumn("dateOnly", to_date(col("localDateTime")))
rap_flat_combined = rap_flat_combined.withColumn("hour", hour(col("localDateTime")))
periodFunc = udf(lambda x: x//time_interval * time_interval)
rap_flat_combined = rap_flat_combined.withColumn("period", periodFunc(col('hour')))
rap_flat_combined.count()

CPU times: user 7.02 ms, sys: 5.77 ms, total: 12.8 ms
Wall time: 15.7 s


16321200

In [13]:
rap_flat_combined.show(5)

+-------------------+---+---+-----------+------------------+-----------------+-----------+------------------+------------------+-------------------+----------+----+------+
|           dateTime| Nx| Ny|iSSRs200430|    meanTemp200430| meanRH_ice200430|iSSRs300370|    meanTemp300370|  meanRH_ice300370|      localDateTime|  dateOnly|hour|period|
+-------------------+---+---+-----------+------------------+-----------------+-----------+------------------+------------------+-------------------+----------+----+------+
|2020-06-01 00:00:00| 44|173|          3|232.63833333333332|73.80749999999999|          2|222.78725000000003|            69.685|2020-05-31 16:00:00|2020-05-31|  16|    12|
|2020-06-01 00:00:00| 54|149|          0| 233.4455833333333|68.72916666666667|          0|          224.5215| 91.44749999999999|2020-05-31 16:00:00|2020-05-31|  16|    12|
|2020-06-01 00:00:00| 56|176|          0|233.02383333333333|          34.3625|          0|         223.17775|50.855000000000004|2020-05-31 1

centerName = center_name.lower()
path = "s3a://partly-cloudy-common-area/" + centerName + "_rap_flat_combined.parquet"
rap_flat_combined.write.parquet(path, mode= "overwrite", compression= 'gzip')

centerName = center_name.lower()
path = "s3a://partly-cloudy-ml-inputs-parquet/" + centerName + "_flat200430.parquet"
rap_flat200430.write.parquet(path, mode= "overwrite", compression= 'gzip')

In [14]:
%%time
rap_flat_combined_pd = rap_flat_combined.groupBy("dateOnly", "period", "Nx", "Ny")\
                                        .agg(sum("iSSRs200430").alias("pdISSRs200430"),
                                             sum("iSSRs300370").alias("pdISSRs300370"),
                                             mean("meanTemp200430").alias("pdMeanTemp200430"),
                                             mean("meanTemp300370").alias("pdMeanTemp300370"),
                                             mean("meanRH_ice200430").alias("pdMeanRH_ice200430"),
                                             mean("meanRH_ice300370").alias("pdMeanRH_ice300370")
                                            )

rap_flat_combined_pd.count()

CPU times: user 37.7 ms, sys: 9.93 ms, total: 47.6 ms
Wall time: 14.8 s


2727704

%%time
rap_flat_combined_pd = rap_flat300370.groupBy(col("dateOnly"), col("period"), col("Nx"), col("Ny"))\
                                        .agg(#sum("iSSRs200430").alias("pdISSRs200430"),
                                            sum("iSSRs300370").alias("pdISSRs300370"),
                                            #mean("meanTemp200430").alias("pdMeanTemp200430"),
                                            mean("meanTemp300370").alias("pdMeanTemp300370"),
                                            #mean("meanRH_ice200430").alias("pdMeanRH_ice200430"),
                                            mean("meanRH_ice300370").alias("pdMeanRH_ice300370")
                                            )

rap_flat_combined_pd.count()

centerName = center_name.lower()
path = "s3a://partly-cloudy-common-area/" + centerName + "_6_hour_flat300370.parquet"
rap_flat_combined_pd.write.parquet(path, mode= "overwrite", compression= 'gzip')

%%time
centerName = center_name.lower()
rap_flat_combined_pd = spark.read.format('parquet')\
                            .load("s3a://partly-cloudy-common-area/" + centerName + "_6_hour_flat.parquet")


In [15]:
%%time
rap_flat_combined_pdPandas = rap_flat_combined_pd.toPandas()
rap_flat_combined_pdPandas

CPU times: user 22.1 s, sys: 888 ms, total: 23 s
Wall time: 39.9 s


,dateOnly,period,Nx,Ny,pdISSRs200430,pdISSRs300370,pdMeanTemp200430,pdMeanTemp300370,pdMeanRH_ice200430,pdMeanRH_ice300370
0,2020-06-01,12,56,172,0,0,232.429806,224.851542,33.503194,22.165833
1,2020-06-01,18,41,160,0,0,234.239861,226.558917,51.325139,56.227083
2,2020-06-02,6,52,141,0,0,235.578000,226.726458,46.706111,65.731667
3,2020-06-03,12,35,149,0,0,235.709181,227.636458,68.600833,72.395417
4,2020-06-04,0,61,133,0,0,235.909153,227.487208,67.727778,75.655000
...,...,...,...,...,...,...,...,...,...,...
2727699,2020-09-05,18,56,159,0,0,240.100792,231.288208,32.444722,61.034583
2727700,2020-07-24,18,79,164,0,0,235.787458,227.110417,38.961389,57.812083
2727701,2021-01-31,12,51,167,0,0,226.920083,218.215000,68.175417,55.083750
2727702,2021-02-09,18,53,169,0,0,227.066139,217.599833,51.715694,56.144167


In [16]:
%%time
rap_flat_combined_pdPandas['LocalDateTimePeriod'] = [pd.Timestamp(rap_flat_combined_pdPandas['dateOnly'][i]) + timedelta(hours= int(rap_flat_combined_pdPandas['period'][i])) for i in range(len(rap_flat_combined_pdPandas))]
rap_flat_combined_pdPandas

CPU times: user 1min 6s, sys: 0 ns, total: 1min 6s
Wall time: 1min 6s


,dateOnly,period,Nx,Ny,pdISSRs200430,pdISSRs300370,pdMeanTemp200430,pdMeanTemp300370,pdMeanRH_ice200430,pdMeanRH_ice300370,LocalDateTimePeriod
0,2020-06-01,12,56,172,0,0,232.429806,224.851542,33.503194,22.165833,2020-06-01 12:00:00
1,2020-06-01,18,41,160,0,0,234.239861,226.558917,51.325139,56.227083,2020-06-01 18:00:00
2,2020-06-02,6,52,141,0,0,235.578000,226.726458,46.706111,65.731667,2020-06-02 06:00:00
3,2020-06-03,12,35,149,0,0,235.709181,227.636458,68.600833,72.395417,2020-06-03 12:00:00
4,2020-06-04,0,61,133,0,0,235.909153,227.487208,67.727778,75.655000,2020-06-04 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...
2727699,2020-09-05,18,56,159,0,0,240.100792,231.288208,32.444722,61.034583,2020-09-05 18:00:00
2727700,2020-07-24,18,79,164,0,0,235.787458,227.110417,38.961389,57.812083,2020-07-24 18:00:00
2727701,2021-01-31,12,51,167,0,0,226.920083,218.215000,68.175417,55.083750,2021-01-31 12:00:00
2727702,2021-02-09,18,53,169,0,0,227.066139,217.599833,51.715694,56.144167,2021-02-09 18:00:00


In [17]:
%%time
rap_flat_combined_pdPandas.drop(columns= ['dateOnly', 'period'], inplace= True)

CPU times: user 175 ms, sys: 61.2 ms, total: 236 ms
Wall time: 235 ms


In [18]:
%%time
centers_df = wr.s3.read_csv('s3://partly-cloudy-common-area/proof_of_concept/rapCellNxNy_ARTCC_intersect.csv')
centers_df

CPU times: user 109 ms, sys: 15.9 ms, total: 125 ms
Wall time: 257 ms


,IDENT,NAME,Nx,Ny
0,ZMA,MIAMI,233,28
1,ZMA,MIAMI,242,29
2,ZMA,MIAMI,232,28
3,ZMA,MIAMI,241,29
4,ZMA,MIAMI,249,30
...,...,...,...,...
25318,ZSE,SEATTLE,76,173
25319,ZLC,SALT LAKE CITY,114,168
25320,ZLC,SALT LAKE CITY,126,167
25321,ZMP,MINNEAPOLIS,144,166


In [19]:
%%time
latlon_df = wr.s3.read_csv('s3://partly-cloudy-common-area/proof_of_concept/latlonRAP.csv')
latlon_df

CPU times: user 136 ms, sys: 12.2 ms, total: 148 ms
Wall time: 253 ms


,Nx,Ny,Lat,Lon
0,1,1,16.281000,-126.138000
1,2,1,16.322011,-125.954684
2,3,1,16.362789,-125.771252
3,4,1,16.403332,-125.587705
4,5,1,16.443642,-125.404045
...,...,...,...,...
67720,297,225,55.648911,-58.431595
67721,298,225,55.607604,-58.167947
67722,299,225,55.565986,-57.904583
67723,300,225,55.524058,-57.641507


In [20]:
%%time
center_flat_pdPandas_join = pd.merge(rap_flat_combined_pdPandas, centers_df, on= ['Nx', 'Ny'])
center_flat_pdPandas_join = pd.merge(center_flat_pdPandas_join, latlon_df, on= ['Nx', 'Ny'])
center_flat_pdPandas_join

CPU times: user 692 ms, sys: 108 ms, total: 800 ms
Wall time: 800 ms


,Nx,Ny,pdISSRs200430,pdISSRs300370,pdMeanTemp200430,pdMeanTemp300370,pdMeanRH_ice200430,pdMeanRH_ice300370,LocalDateTimePeriod,IDENT,NAME,Lat,Lon
0,56,172,0,0,232.429806,224.851542,33.503194,22.165833,2020-06-01 12:00:00,ZSE,SEATTLE,48.222044,-122.476056
1,56,172,1,1,234.362222,226.758125,67.710833,80.281250,2020-10-14 06:00:00,ZSE,SEATTLE,48.222044,-122.476056
2,56,172,0,0,226.890944,217.765542,59.252500,57.524167,2021-03-23 06:00:00,ZSE,SEATTLE,48.222044,-122.476056
3,56,172,0,0,235.005333,226.913125,54.848611,68.529167,2020-11-02 00:00:00,ZSE,SEATTLE,48.222044,-122.476056
4,56,172,0,0,234.989014,227.302500,47.583889,38.764583,2020-08-06 06:00:00,ZSE,SEATTLE,48.222044,-122.476056
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2727699,41,164,1,1,233.592569,224.732542,76.104444,82.351250,2020-10-12 06:00:00,ZSE,SEATTLE,46.371628,-125.686377
2727700,41,164,2,2,236.605625,228.884625,77.633750,91.970000,2020-06-22 12:00:00,ZSE,SEATTLE,46.371628,-125.686377
2727701,41,164,0,0,238.268500,230.716083,38.629444,47.003750,2020-07-20 06:00:00,ZSE,SEATTLE,46.371628,-125.686377
2727702,41,164,0,0,237.249319,228.383750,60.181250,65.685417,2020-07-28 18:00:00,ZSE,SEATTLE,46.371628,-125.686377


In [21]:
%%time
center_flat_pdPandas_join['geoEncode'] = [pgh.encode(center_flat_pdPandas_join['Lat'][i], center_flat_pdPandas_join['Lon'][i], precision= 5) for i in range(len(center_flat_pdPandas_join))]
center_flat_pdPandas_join

CPU times: user 2min 3s, sys: 89.8 ms, total: 2min 3s
Wall time: 2min 3s


,Nx,Ny,pdISSRs200430,pdISSRs300370,pdMeanTemp200430,pdMeanTemp300370,pdMeanRH_ice200430,pdMeanRH_ice300370,LocalDateTimePeriod,IDENT,NAME,Lat,Lon,geoEncode
0,56,172,0,0,232.429806,224.851542,33.503194,22.165833,2020-06-01 12:00:00,ZSE,SEATTLE,48.222044,-122.476056,c28fk
1,56,172,1,1,234.362222,226.758125,67.710833,80.281250,2020-10-14 06:00:00,ZSE,SEATTLE,48.222044,-122.476056,c28fk
2,56,172,0,0,226.890944,217.765542,59.252500,57.524167,2021-03-23 06:00:00,ZSE,SEATTLE,48.222044,-122.476056,c28fk
3,56,172,0,0,235.005333,226.913125,54.848611,68.529167,2020-11-02 00:00:00,ZSE,SEATTLE,48.222044,-122.476056,c28fk
4,56,172,0,0,234.989014,227.302500,47.583889,38.764583,2020-08-06 06:00:00,ZSE,SEATTLE,48.222044,-122.476056,c28fk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2727699,41,164,1,1,233.592569,224.732542,76.104444,82.351250,2020-10-12 06:00:00,ZSE,SEATTLE,46.371628,-125.686377,c0nxg
2727700,41,164,2,2,236.605625,228.884625,77.633750,91.970000,2020-06-22 12:00:00,ZSE,SEATTLE,46.371628,-125.686377,c0nxg
2727701,41,164,0,0,238.268500,230.716083,38.629444,47.003750,2020-07-20 06:00:00,ZSE,SEATTLE,46.371628,-125.686377,c0nxg
2727702,41,164,0,0,237.249319,228.383750,60.181250,65.685417,2020-07-28 18:00:00,ZSE,SEATTLE,46.371628,-125.686377,c0nxg


In [22]:
%%time
center_flat_pd_final = center_flat_pdPandas_join[['LocalDateTimePeriod', 
                                                  'Nx', 'Ny', 
                                                  'Lat', 'Lon', 
                                                  'geoEncode', 
                                                  'NAME', 
                                                  'pdISSRs200430', 
                                                  'pdISSRs300370',
                                                  'pdMeanTemp200430',
                                                  'pdMeanTemp300370',
                                                  'pdMeanRH_ice200430',
                                                  'pdMeanRH_ice300370']].copy()
center_flat_pd_final

CPU times: user 378 ms, sys: 180 ms, total: 558 ms
Wall time: 557 ms


,LocalDateTimePeriod,Nx,Ny,Lat,Lon,geoEncode,NAME,pdISSRs200430,pdISSRs300370,pdMeanTemp200430,pdMeanTemp300370,pdMeanRH_ice200430,pdMeanRH_ice300370
0,2020-06-01 12:00:00,56,172,48.222044,-122.476056,c28fk,SEATTLE,0,0,232.429806,224.851542,33.503194,22.165833
1,2020-10-14 06:00:00,56,172,48.222044,-122.476056,c28fk,SEATTLE,1,1,234.362222,226.758125,67.710833,80.281250
2,2021-03-23 06:00:00,56,172,48.222044,-122.476056,c28fk,SEATTLE,0,0,226.890944,217.765542,59.252500,57.524167
3,2020-11-02 00:00:00,56,172,48.222044,-122.476056,c28fk,SEATTLE,0,0,235.005333,226.913125,54.848611,68.529167
4,2020-08-06 06:00:00,56,172,48.222044,-122.476056,c28fk,SEATTLE,0,0,234.989014,227.302500,47.583889,38.764583
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2727699,2020-10-12 06:00:00,41,164,46.371628,-125.686377,c0nxg,SEATTLE,1,1,233.592569,224.732542,76.104444,82.351250
2727700,2020-06-22 12:00:00,41,164,46.371628,-125.686377,c0nxg,SEATTLE,2,2,236.605625,228.884625,77.633750,91.970000
2727701,2020-07-20 06:00:00,41,164,46.371628,-125.686377,c0nxg,SEATTLE,0,0,238.268500,230.716083,38.629444,47.003750
2727702,2020-07-28 18:00:00,41,164,46.371628,-125.686377,c0nxg,SEATTLE,0,0,237.249319,228.383750,60.181250,65.685417


In [23]:
output_bucket = 'partly-cloudy-ml-inputs'
outputFileName = center_name + '_6_hour_flat.csv' 

In [24]:
wr.s3.to_csv(center_flat_pd_final, f"s3://{output_bucket}/{outputFileName}", index=False)

{'paths': ['s3://partly-cloudy-ml-inputs/SEATTLE_6_hour_flat.csv'],
 'partitions_values': {}}